# 第 14 期 Datawhale 组队学习

## 一、端午节的淘宝粽子交易

In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.__version__

'1.0.5'

In [3]:
df = pd.read_csv('端午粽子数据.csv').convert_dtypes()

In [4]:
df.head()

,标题,价格,付款人数,店铺,发货地址
0,五芳斋粽子礼盒 心悦+18只装咸鸭蛋组合端午节礼品团购嘉兴肉粽子,129,6人付款,五芳斋官方旗舰店,浙江 嘉兴
1,北京稻香村端午粽子手工豆沙粽220g*2袋散装豆沙粽香甜软糯豆沙粽,44,8人付款,天猫超市,上海
2,五芳斋粽子礼盒装鲜肉咸蛋黄大肉粽嘉兴豆沙甜粽端午团购散装礼品,89.9,100万+人付款,五芳斋官方旗舰店,浙江 嘉兴
3,稻香私房鲜肉粽蛋黄肉粽嘉兴粽子咸鸭蛋礼盒装端午节送礼特产团购,138,1936人付款,稻香村食品旗舰店,北京
4,嘉兴粽子 蛋黄鲜肉粽新鲜大肉粽早餐散装团购浙江特产蜜枣多口味,3.8,9500+人付款,城城喂食猫,浙江 嘉兴


In [5]:
df.columns

Index(['标题', ' 价格', '付款人数', '店铺', '发货地址 '], dtype='object')

In [6]:
df = df.rename(columns={' 价格':'价格', '发货地址 ':'发货地址'})
df.columns

Index(['标题', '价格', '付款人数', '店铺', '发货地址'], dtype='object')

### (1) 请删除最后一列为缺失值的行，并求所有在杭州发货的商品单价均值。

In [7]:
# df['价格'] = pd.to_numeric(df['价格'], errors='coerce')

In [8]:
df['价格'][~(df['价格'].str.replace(r'-?\d+\.?\d*','True')=='True')]

538     45.9.
4376      45_
Name: 价格, dtype: string

In [9]:
df.loc[[538, 4376], '价格']=['45.9', '45']

In [10]:
d2 = df[df['发货地址'].notnull()]

In [11]:
df2['价格'][df['发货地址'].str.contains('杭州')].astype(float).mean()

NameError: name 'df2' is not defined

### (2) 商品标题带有“嘉兴”但发货地却不在嘉兴的商品有多少条记录？

In [12]:
df[df['标题'].str.contains('嘉兴') & ~df['发货地址'].str.contains('嘉兴')].shape[0]

1032

### (3) 请按照分位数将价格分为“高、较高、中、较低、低” 5 个类别，再将类别结果插入到标题一列之后，最后对类别列进行降序排序。

In [13]:
df['类别'] = pd.qcut(df['价格'].astype(float),[0,0.2,0.4,0.6,0.8,1.],labels=['低','较低','中','较高','高'])
df.head()

,标题,价格,付款人数,店铺,发货地址,类别
0,五芳斋粽子礼盒 心悦+18只装咸鸭蛋组合端午节礼品团购嘉兴肉粽子,129,6人付款,五芳斋官方旗舰店,浙江 嘉兴,高
1,北京稻香村端午粽子手工豆沙粽220g*2袋散装豆沙粽香甜软糯豆沙粽,44,8人付款,天猫超市,上海,中
2,五芳斋粽子礼盒装鲜肉咸蛋黄大肉粽嘉兴豆沙甜粽端午团购散装礼品,89.9,100万+人付款,五芳斋官方旗舰店,浙江 嘉兴,较高
3,稻香私房鲜肉粽蛋黄肉粽嘉兴粽子咸鸭蛋礼盒装端午节送礼特产团购,138,1936人付款,稻香村食品旗舰店,北京,高
4,嘉兴粽子 蛋黄鲜肉粽新鲜大肉粽早餐散装团购浙江特产蜜枣多口味,3.8,9500+人付款,城城喂食猫,浙江 嘉兴,低


In [14]:
df3 = df.reindex(columns=['标题','类别','价格','付款人数','店铺','发货地址']).sort_values('类别',ascending=False)
df3.head()

,标题,类别,价格,付款人数,店铺,发货地址
0,五芳斋粽子礼盒 心悦+18只装咸鸭蛋组合端午节礼品团购嘉兴肉粽子,高,129,6人付款,五芳斋官方旗舰店,浙江 嘉兴
847,真真老老蛋黄肉粽140g*60个新鲜大肉粽冷冻方便早餐嘉兴粽子端午,高,168,139人付款,tb6182109_2012,上海
2773,山西双合成粽子端午节送礼御粽膳坊礼盒粽叶板栗豆沙黄米杂粮粽,高,129,33人付款,双合成旗舰店,山西 晋中
2768,采芝斋粽子甜粽新鲜蛋黄大肉粽嘉兴口味鲜肉粽子食品端午礼盒定制,高,216,47人付款,采芝斋杭州专卖店,浙江 杭州
2763,真真老老臻瑞礼盒1.12kg/盒端午节粽子礼盒装送130g经典鲜肉粽,高,126,2人付款,天猫超市,上海


### (4) 付款人数一栏有缺失值吗？若有则请利用上一问的分类结果对这些缺失值进行合理估计并填充。

In [15]:
df3[df3['付款人数'].isnull()]

,标题,类别,价格,付款人数,店铺,发货地址
595,稻香村粽子礼盒京彩缤纷1670g粽子松仁肉肚鸭蛋 甜粽肉粽多口味,高,189,<NA>,天猫会员店,上海
1074,三全龙舟粽礼盒五月尚品礼盒1800g北方端午粽子礼盒,高,129,<NA>,天猫会员店,上海
386,五芳斋华礼竹篮礼盒1360g蛋粽组合端午礼品嘉兴粽子礼盒,高,129,<NA>,天猫会员店,上海
491,久知味双层竹篮粽子酱鸭礼盒1560克 蛋黄粽粽子礼盒装 端午节送礼,较高,89,<NA>,天猫会员店,上海
438,真真老老嘉情礼盒10粽6蛋1.52kg/盒嘉兴粽子端午粽子礼盒,较高,69.9,<NA>,天猫会员店,上海
...,...,...,...,...,...,...
756,五芳斋嘉兴粽子真空散装大肉粽140g*2只端午特产早餐速食,低,18.6,<NA>,天猫会员店,上海
1813,真真老老 嘉兴 栗子肉粽 130g 粽子 真空 端午节,低,4.5,<NA>,天猫会员店,上海
1812,真真老老 嘉兴 豆沙粽子 130g 真空 粽子 端午节,低,3.9,<NA>,天猫会员店,上海
2130,三珍斋蛋黄粽200g真空100g*2只嘉兴特产粽子速食早餐,低,7.88,<NA>,天猫会员店,上海


In [16]:
df4 = df3[df['付款人数'].notnull()]
df4

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,标题,类别,价格,付款人数,店铺,发货地址
0,五芳斋粽子礼盒 心悦+18只装咸鸭蛋组合端午节礼品团购嘉兴肉粽子,高,129,6人付款,五芳斋官方旗舰店,浙江 嘉兴
847,真真老老蛋黄肉粽140g*60个新鲜大肉粽冷冻方便早餐嘉兴粽子端午,高,168,139人付款,tb6182109_2012,上海
2773,山西双合成粽子端午节送礼御粽膳坊礼盒粽叶板栗豆沙黄米杂粮粽,高,129,33人付款,双合成旗舰店,山西 晋中
2768,采芝斋粽子甜粽新鲜蛋黄大肉粽嘉兴口味鲜肉粽子食品端午礼盒定制,高,216,47人付款,采芝斋杭州专卖店,浙江 杭州
2763,真真老老臻瑞礼盒1.12kg/盒端午节粽子礼盒装送130g经典鲜肉粽,高,126,2人付款,天猫超市,上海
...,...,...,...,...,...,...
2126,真真老老方便速食即食肉粽130g*2嘉兴粽子特产特色方便早餐,低,9.9,77人付款,天猫超市,上海
2127,荃盛粽子蜜枣130*2真空包装甜粽嘉兴粽端午美味早餐,低,9.9,328人付款,天猫超市,上海
3637,新鲜粽叶包粽子的叶子干粽子叶大号竹子叶竹叶苇叶皮大鲜粽叶干种,低,4,617人付款,海倪食品专营店,广东 广州
2129,嘉兴五芳斋粽子 真空100克*2只紫米栗蓉粽端午节甜粽子,低,11,2897人付款,五芳斋官方旗舰店,浙江 嘉兴


In [17]:
s = df4['付款人数']
s = s.str.replace('人付款', '').str.replace('+', '')
# s = s.str.replace('人付款', '').str.replace('万', '0000').str.replace('+', '')
# s.astype('int')
s1 = s[s.str.contains('万')].str.replace('万', '').astype(float)*10000
s2 = s[~s.str.contains('万')].astype(int)
s = pd.concat([s1, s2])
s.name='付款人数'
s.head()

1333    10000.0
11      20000.0
804     65000.0
104     15000.0
26      85000.0
Name: 付款人数, dtype: float64

In [18]:
df3.update(s)
df3.head()

,标题,类别,价格,付款人数,店铺,发货地址
0,五芳斋粽子礼盒 心悦+18只装咸鸭蛋组合端午节礼品团购嘉兴肉粽子,高,129,6,五芳斋官方旗舰店,浙江 嘉兴
847,真真老老蛋黄肉粽140g*60个新鲜大肉粽冷冻方便早餐嘉兴粽子端午,高,168,139,tb6182109_2012,上海
2773,山西双合成粽子端午节送礼御粽膳坊礼盒粽叶板栗豆沙黄米杂粮粽,高,129,33,双合成旗舰店,山西 晋中
2768,采芝斋粽子甜粽新鲜蛋黄大肉粽嘉兴口味鲜肉粽子食品端午礼盒定制,高,216,47,采芝斋杭州专卖店,浙江 杭州
2763,真真老老臻瑞礼盒1.12kg/盒端午节粽子礼盒装送130g经典鲜肉粽,高,126,2,天猫超市,上海


In [22]:
df5 = df3.groupby('类别').apply(lambda x: x['付款人数'].fillna(x['付款人数'].mean())).reset_index(0)
df5.head()

,类别,付款人数
3956,低,292.0
2705,低,91.0
2649,低,423.0
1064,低,368.0
881,低,847.0


In [23]:
df6 = df5.join(df4,rsuffix='_').reindex(columns=['标题','类别','价格','付款人数','店铺','发货地址'])
df6['付款人数'] = df6['付款人数'].astype(int)
df6.head()

,标题,类别,价格,付款人数,店铺,发货地址
3956,新鲜粽叶现摘大号粽子叶江西省农家野生干箬叶5斤特级广西大棕叶,低,11.36,292,毛丽兰87,江西 吉安
2705,嘉兴特产粽子咸鸭蛋礼盒装新鲜大肉粽端午节礼品粽子礼盒团购送礼,低,18.9,91,qixiaoxia8811,江苏 无锡
2649,粽叶干大号免邮广东省广西野生粑棕子叶新鲜包粽子叶芦苇叶大柊叶,低,9.9,423,秋与页,广东 广州
1064,采芝斋 蛋黄肉粽200g真空包装嘉兴粽子百年工艺送礼佳品速食早餐,低,16.9,368,天猫超市,上海
881,傅太粽子嘉兴风味新鲜杂粮散装甜糯米手工八宝粽礼袋包装素粽批发,低,22.8,847,傅太旗舰店,江西 萍乡


### (5) 请将数据后四列合并为如下格式的 Series：商品发货地为 ××，店铺为××，共计 ×× 人付款，单价为 ××。

In [24]:
s5 = '商品发货地为 '+df6['发货地址']+', 店铺为'+df6['店铺']+', 共计 '+df6['付款人数'].astype('int').astype('str')+'人付款, 单价为'+df6['价格'].astype('str')
s5

3956          商品发货地为 江西 吉安, 店铺为毛丽兰87, 共计 292人付款, 单价为11.36
2705    商品发货地为 江苏 无锡, 店铺为qixiaoxia8811, 共计 91人付款, 单价为18.9
2649              商品发货地为 广东 广州, 店铺为秋与页, 共计 423人付款, 单价为9.9
1064               商品发货地为 上海, 店铺为天猫超市, 共计 368人付款, 单价为16.9
881            商品发货地为 江西 萍乡, 店铺为傅太旗舰店, 共计 847人付款, 单价为22.8
                              ...                        
88           商品发货地为 浙江 嘉兴, 店铺为三珍斋禧氏专卖店, 共计 746人付款, 单价为118
4296               商品发货地为 海南 定安, 店铺为嘉麟尚品, 共计 9人付款, 单价为159
3603       商品发货地为 海南 儋州, 店铺为正佰晨食品专营店, 共计 17人付款, 单价为116.13
61          商品发货地为 浙江 嘉兴, 店铺为五芳斋官方旗舰店, 共计 3817人付款, 单价为189
4294                商品发货地为 浙江 杭州, 店铺为小楚, 共计 18人付款, 单价为128
Length: 4403, dtype: string

### (6) 请将上一问中的结果恢复成原来的四列。

In [25]:
df7 =  s5.str.extract(r'商品发货地为(?P<发货地址>.+), 店铺为(?P<店铺>.+), 共计 (?P<付款人数>.+)人付款, 单价为(?P<价格>.+)')
df7

,发货地址,店铺,付款人数,价格
3956,江西 吉安,毛丽兰87,292,11.36
2705,江苏 无锡,qixiaoxia8811,91,18.9
2649,广东 广州,秋与页,423,9.9
1064,上海,天猫超市,368,16.9
881,江西 萍乡,傅太旗舰店,847,22.8
...,...,...,...,...
88,浙江 嘉兴,三珍斋禧氏专卖店,746,118
4296,海南 定安,嘉麟尚品,9,159
3603,海南 儋州,正佰晨食品专营店,17,116.13
61,浙江 嘉兴,五芳斋官方旗舰店,3817,189


## 二、墨尔本每日最低温度

In [26]:
d = pd.read_csv(r'墨尔本温度数据.csv')
d.Date = d.Date.astype('datetime64[ns]')
d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3650 entries, 0 to 3649
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    3650 non-null   datetime64[ns]
 1   Temp    3650 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 57.2 KB


### (1) 剔除国庆节、五一劳动节和每月第一个周一，求每月的平均最低气温。

In [27]:
d10=d[((d.Date.dt.month==5) | (d.Date.dt.month==10)) & (d.Date.dt.day==1)]
d10

,Date,Temp
120,1981-05-01,8.6
273,1981-10-01,8.9
485,1982-05-01,7.5
638,1982-10-01,8.6
850,1983-05-01,12.5
1003,1983-10-01,9.3
1216,1984-05-01,13.8
1369,1984-10-01,11.3
1580,1985-05-01,9.4
1733,1985-10-01,10.5


In [28]:
d['year'] = d.Date.apply(lambda x:x.year)
d['month'] = d.Date.apply(lambda x:x.month)
d['day'] = d.Date.apply(lambda x:x.day)
d['weekday'] = d.Date.apply(lambda x:x.weekday())
d

,Date,Temp,year,month,day,weekday
0,1981-01-01,20.7,1981,1,1,3
1,1981-01-02,17.9,1981,1,2,4
2,1981-01-03,18.8,1981,1,3,5
3,1981-01-04,14.6,1981,1,4,6
4,1981-01-05,15.8,1981,1,5,0
...,...,...,...,...,...,...
3645,1990-12-27,14.0,1990,12,27,3
3646,1990-12-28,13.6,1990,12,28,4
3647,1990-12-29,13.5,1990,12,29,5
3648,1990-12-30,15.7,1990,12,30,6


In [29]:
d11=d.groupby(['year','month']).apply(lambda x: x[x.weekday==0].sort_values(by='day',ascending=True).head(1)).droplevel([0,1])
d11

,Date,Temp,year,month,day,weekday
4,1981-01-05,15.8,1981,1,5,0
32,1981-02-02,18.8,1981,2,2,0
60,1981-03-02,18.6,1981,3,2,0
95,1981-04-06,16.6,1981,4,6,0
123,1981-05-04,8.6,1981,5,4,0
...,...,...,...,...,...,...
3502,1990-08-06,7.8,1990,8,6,0
3530,1990-09-03,8.6,1990,9,3,0
3558,1990-10-01,11.3,1990,10,1,0
3593,1990-11-05,12.5,1990,11,5,0


In [30]:
d.iloc[[i for i in d.index if i not in d10.index and i not in d11.index],[0,1,2,3]].groupby(['year','month'])['Temp'].min()


year  month
1981  1        12.1
      2        13.7
      3         8.0
      4         7.4
      5         2.1
               ... 
1990  8         2.1
      9         3.0
      10        6.0
      11        8.3
      12       10.0
Name: Temp, Length: 120, dtype: float64

### (2) 季节指数是一种对于周期性变化序列的特征刻画。
记数据集中第 $k$ 年平均最低气温为 $TY_k (k = 1,...,10)$，第 $k$ 年第 $j$ 个月平均最低气温为$TM_{kj} (j = 1,...,12)$，定义 
$$S_j =\frac{∑_k TM_{kj}}{∑_k TY_k}$$
请按照如上定义，计算 12 个月的季节指数 $S_j$ 。

In [31]:
TY_k = d.groupby('year')['Temp'].min()
TY_k

year
1981    2.1
1982    0.0
1983    0.0
1984    0.1
1985    0.3
1986    0.8
1987    1.5
1988    2.8
1989    0.5
1990    2.1
Name: Temp, dtype: float64

In [32]:
TM_kj = d.groupby(['year','month'])['Temp'].min()
TM_kj

year  month
1981  1        12.1
      2        13.7
      3         8.0
      4         7.4
      5         2.1
               ... 
1990  8         2.1
      9         3.0
      10        6.0
      11        8.3
      12       10.0
Name: Temp, Length: 120, dtype: float64

In [33]:
TM_kj.unstack().sum()/TY_k.sum()

month
1     10.401961
2     11.039216
3      8.911765
4      6.921569
5      4.392157
6      1.725490
7      1.794118
8      3.196078
9      4.450980
10     5.274510
11     7.352941
12     9.549020
dtype: float64

### (3) 移动平均法是一种时间序列的常见平滑方式，可分为 k 期移动平均和 k期中心移动平均，都使用了某一时刻及其周围的数据对该时刻的数据进行平滑修正。
设原序列为 $x_1 ,...,x_n$ ，对于 $x_t$ 的 $k$ 期移动平均修正 $\tilde{x_t}$ 为
$$\frac{∑^{k−1}_{i=0}x_{t−i}}{k}$$
，对于 k 期中心移动平均修正为

$$
\tilde{x_t} =
\begin{cases}
\frac{1}{k}(\frac{1}{2}x_{t-\frac{k}{2}}+x_{t-\frac{k}{2}+1}+...+x_{t}+...+x_{t+\frac{k}{2}-1}+\frac{1}{2}x_{t+\frac{k}{2}}) & \text{ , k 为偶数 } \\
\frac{1}{k}(x_{t-\frac{k-1}{2}}+x_{t-\frac{k-1}{2}+1}+...+x_{t}+...+x_{t+\frac{k-1}{2}-1}+x_{t+\frac{k-1}{2}}) & \text{ , k 为奇数 } 
\end{cases} 
$$

(a) 求原序列的 5 期移动平均序列。   
(b) 求原序列的 5 期与 6 期中心移动平均序列。

In [34]:
# (a)
d[['Date','Temp']].set_index('Date').rolling(window=5).mean()

,Temp
Date,
1981-01-01,NaN
1981-01-02,NaN
1981-01-03,NaN
1981-01-04,NaN
1981-01-05,17.56
...,...
1990-12-27,13.08
1990-12-28,13.02
1990-12-29,13.72


In [35]:
# (b) 5
d3 = d[['Date','Temp']].set_index('Date')
d3.rolling(window=5).apply(lambda x:x.sum()/5).shift(-2)

,Temp
Date,
1981-01-01,NaN
1981-01-02,NaN
1981-01-03,17.56
1981-01-04,16.58
1981-01-05,16.16
...,...
1990-12-27,13.72
1990-12-28,14.28
1990-12-29,13.96


In [36]:
# (b) 6
d3.rolling(window=7).apply(lambda x:(x.sum()-x[0]/2-x[6]/2)/6).shift(-3)

,Temp
Date,
1981-01-01,NaN
1981-01-02,NaN
1981-01-03,NaN
1981-01-04,16.858333
1981-01-05,16.408333
...,...
1990-12-27,13.575000
1990-12-28,14.058333
1990-12-29,NaN


## 三、 2016 年 8 月上海市摩拜单车骑行记录

In [37]:
from datetime import datetime
df = pd.read_csv(r'摩拜单车数据.csv')
df.start_time = df.start_time.apply(lambda x:datetime.strptime(str(x),'%Y-%m-%d %H:%M'))
df.end_time = df.end_time.apply(lambda x:datetime.strptime(str(x),'%Y-%m-%d %H:%M'))
df
df.head()

,orderid,bikeid,userid,start_time,start_location_x,start_location_y,end_time,end_location_x,end_location_y,track
0,78387,158357,10080,2016-08-20 06:57:00,121.348,31.389,2016-08-20 07:04:00,121.357,31.388,"121.347,31.392#121.348,31.389#121.349,31.390#1..."
1,891333,92776,6605,2016-08-29 19:09:00,121.508,31.279,2016-08-29 19:31:00,121.489,31.271,"121.489,31.270#121.489,31.271#121.490,31.270#1..."
2,1106623,152045,8876,2016-08-13 16:17:00,121.383,31.254,2016-08-13 16:36:00,121.405,31.248,"121.381,31.251#121.382,31.251#121.382,31.252#1..."
3,1389484,196259,10648,2016-08-23 21:34:00,121.484,31.320,2016-08-23 21:43:00,121.471,31.325,"121.471,31.325#121.472,31.325#121.473,31.324#1..."
4,188537,78208,11735,2016-08-16 07:32:00,121.407,31.292,2016-08-16 07:41:00,121.418,31.288,"121.407,31.291#121.407,31.292#121.408,31.291#1..."


### (1) 平均而言，周末单天用车量比工作日单天用车量更大吗？

In [38]:
df['weekday'] = df.start_time.apply(lambda x: x.weekday())
df.weekday.value_counts().sort_index()

0    16380
1    16259
2    17829
3    12484
4    13208
5    13359
6    12842
Name: weekday, dtype: int64

### (2) 工作日每天的高峰时间段大致为上午 7:30 至 9:30、下午 17:00 至 19:00，请问 8 月里早高峰骑行记录量（以 start_time 为准）高于晚高峰的有几天？

In [39]:
df2 = df.start_time.astype('datetime64[ns]').to_frame()
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102361 entries, 0 to 102360
Data columns (total 1 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   start_time  102361 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 799.8 KB


In [40]:
df2['date'] = df2.start_time.apply(lambda x: x.date())
df2['time'] = df2.start_time.apply(lambda x: x.time())
df2

,start_time,date,time
0,2016-08-20 06:57:00,2016-08-20,06:57:00
1,2016-08-29 19:09:00,2016-08-29,19:09:00
2,2016-08-13 16:17:00,2016-08-13,16:17:00
3,2016-08-23 21:34:00,2016-08-23,21:34:00
4,2016-08-16 07:32:00,2016-08-16,07:32:00
...,...,...,...
102356,2016-08-28 18:03:00,2016-08-28,18:03:00
102357,2016-08-14 20:22:00,2016-08-14,20:22:00
102358,2016-08-27 09:54:00,2016-08-27,09:54:00
102359,2016-08-20 06:04:00,2016-08-20,06:04:00


In [41]:
s21 = df2[(df2.time>datetime(1,1,1,7,30,0).time()) & (df2.time<datetime(1,1,1,9,30,0).time())].groupby(df2.date)['date'].count()

In [42]:
s22 = df2[(df2.time>datetime(1,1,1,17,0,0).time()) & (df2.time<datetime(1,1,1,19,0,0).time())].groupby(df2.date)['date'].count()

In [43]:
df22 = s21.to_frame().join(s22.to_frame(),lsuffix='_zao',rsuffix='_wan')
df22[df22.date_zao>df22.date_wan]

,date_zao,date_wan
date,,
2016-08-04,412,215
2016-08-29,884,385
2016-08-30,846,561


### (3) 请给出在所有周五中（以 start_time 为准），记录条数最多的那个周五所在的日期，并在该天内分别按 30 分钟、2 小时、6 小时统计摩拜单车使用时间的均值。

In [44]:
# 先增加weekday
df2['weekday'] = df2.start_time.apply(lambda x:x.weekday())
# 找出所有的周五
df2[df2.weekday==4]
# 查看哪个周五记录条数最多
df2[df2.weekday==4].groupby('date').count()
# 筛选出记录条数最多的周五的记录
df3 = df2[df2.date==pd.to_datetime('2016-08-26',format='%Y-%m-%d')]
#.join(df.endtime.astype())
df3

,start_time,date,time,weekday
11,2016-08-26 17:57:00,2016-08-26,17:57:00,4
32,2016-08-26 07:45:00,2016-08-26,07:45:00,4
37,2016-08-26 22:05:00,2016-08-26,22:05:00,4
53,2016-08-26 14:06:00,2016-08-26,14:06:00,4
59,2016-08-26 22:05:00,2016-08-26,22:05:00,4
...,...,...,...,...
102265,2016-08-26 17:56:00,2016-08-26,17:56:00,4
102304,2016-08-26 13:27:00,2016-08-26,13:27:00,4
102313,2016-08-26 20:13:00,2016-08-26,20:13:00,4
102317,2016-08-26 12:43:00,2016-08-26,12:43:00,4


In [45]:
# 增加 end_time 列
df3 = df3.join(df.end_time.astype('datetime64[ns]'))
df3
#.set_index('start_time')

,start_time,date,time,weekday,end_time
11,2016-08-26 17:57:00,2016-08-26,17:57:00,4,2016-08-26 18:43:00
32,2016-08-26 07:45:00,2016-08-26,07:45:00,4,2016-08-26 07:53:00
37,2016-08-26 22:05:00,2016-08-26,22:05:00,4,2016-08-26 22:27:00
53,2016-08-26 14:06:00,2016-08-26,14:06:00,4,2016-08-26 14:15:00
59,2016-08-26 22:05:00,2016-08-26,22:05:00,4,2016-08-26 22:13:00
...,...,...,...,...,...
102265,2016-08-26 17:56:00,2016-08-26,17:56:00,4,2016-08-26 18:02:00
102304,2016-08-26 13:27:00,2016-08-26,13:27:00,4,2016-08-26 13:30:00
102313,2016-08-26 20:13:00,2016-08-26,20:13:00,4,2016-08-26 20:23:00
102317,2016-08-26 12:43:00,2016-08-26,12:43:00,4,2016-08-26 12:45:00


In [46]:
df3['使用时间'] = df3.end_time.apply(lambda x:datetime.strptime(str(x), '%Y-%m-%d %H:%M:%S'))-df3.start_time.apply(lambda x:datetime.strptime(str(x), '%Y-%m-%d %H:%M:%S'))
# df3.end_time-df2.start_time # 为什么直接相减,得到的是 NaT?
df3['使用时间']

11       00:46:00
32       00:08:00
37       00:22:00
53       00:09:00
59       00:08:00
           ...   
102265   00:06:00
102304   00:03:00
102313   00:10:00
102317   00:02:00
102353   00:07:00
Name: 使用时间, Length: 4782, dtype: timedelta64[ns]

In [47]:
# 以 30 分钟的频率进行重采样并计算平均使用时长
df3.set_index('start_time').resample('30min')['使用时间'].apply(lambda x:x.sum()/x.count())

start_time
2016-08-26 00:00:00   00:18:56.250000
2016-08-26 00:30:00   00:26:42.352941
2016-08-26 01:00:00   00:13:27.272727
2016-08-26 01:30:00          00:15:30
2016-08-26 02:00:00   00:17:17.142857
2016-08-26 02:30:00   00:14:07.500000
2016-08-26 03:00:00          00:21:45
2016-08-26 03:30:00          00:30:45
2016-08-26 04:00:00          00:13:00
2016-08-26 04:30:00   00:17:08.571428
2016-08-26 05:00:00          00:11:54
2016-08-26 05:30:00   00:13:04.615384
2016-08-26 06:00:00   00:11:40.909090
2016-08-26 06:30:00   00:12:11.666666
2016-08-26 07:00:00   00:10:35.076923
2016-08-26 07:30:00   00:11:52.826086
2016-08-26 08:00:00   00:12:42.424242
2016-08-26 08:30:00   00:10:57.572254
2016-08-26 09:00:00   00:11:15.114503
2016-08-26 09:30:00   00:10:52.903225
2016-08-26 10:00:00   00:13:05.333333
2016-08-26 10:30:00   00:12:22.222222
2016-08-26 11:00:00   00:10:50.476190
2016-08-26 11:30:00   00:11:31.200000
2016-08-26 12:00:00   00:14:45.974025
2016-08-26 12:30:00   00:13:21.573033
2

In [48]:
# 2 小时
df3.set_index('start_time').resample('2h')['使用时间'].apply(lambda x:x.sum()/x.count())

start_time
2016-08-26 00:00:00   00:19:57.600000
2016-08-26 02:00:00   00:19:18.260869
2016-08-26 04:00:00   00:13:33.529411
2016-08-26 06:00:00   00:11:31.255813
2016-08-26 08:00:00   00:11:35.596638
2016-08-26 10:00:00   00:12:01.914893
2016-08-26 12:00:00   00:13:56.742857
2016-08-26 14:00:00   00:15:43.062200
2016-08-26 16:00:00   00:17:11.023842
2016-08-26 18:00:00   00:19:02.704402
2016-08-26 20:00:00   00:26:00.936037
2016-08-26 22:00:00   00:20:38.835616
Freq: 2H, Name: 使用时间, dtype: timedelta64[ns]

In [49]:
# 6 小时
df3.set_index('start_time').resample('6h')['使用时间'].apply(lambda x:x.sum()/x.count())

start_time
2016-08-26 00:00:00   00:17:47.102803
2016-08-26 06:00:00   00:11:39.846977
2016-08-26 12:00:00   00:16:00.283592
2016-08-26 18:00:00   00:21:39.650238
Freq: 6H, Name: 使用时间, dtype: timedelta64[ns]

### (4) 请自行搜索相关代码或调用库，计算每条记录起点到终点的球面距离。

### (5) 摩拜单车的骑行结束时间是以电子锁关闭的记录时间为准，但有时候用户会忘记关锁，导致骑行时间出现异常。同时，正常人的骑行速度往往大致落在一个合理的区间，请结合上一问中的球面距离和骑行起始、结束时间，找出潜在的异常骑行记录。

### (6) 由于路线的曲折性，起点到终点的球面距离往往不能充分反应行程长度，请利用 track 列的路线坐标数据，计算估计实际骑行距离，并重新仿照上一问的方法找出可能的异常记录。